In [ ]:
import tensorflow as tf
import zipfile as zf
import pathlib
#import google.colab.files as files


local_file = '/tmp/rockpaperscissors.zip'


try:
  extract_files = zf.ZipFile(local_file,'r')
  extract_files.extractall('/tmp/')
  extract_files.close()
  print('Extraksi dataset sukses')

except(FileNotFoundError):
   print('\ndataset tidak ditemukan\n')

   print('Mendownload dataset yang diperlukan')
   extract_files = tf.keras.utils.get_file(origin='https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip',cache_dir='/tmp/',fname='/tmp/rockpaperscissors.zip')
   extract_files = zf.ZipFile(local_file,'r')
   extract_files.extractall('/tmp/')
   extract_files.close()

   print('\nExtraksi  ulang dataset sukses')
except:
  print('tidak dapat melakukan Extraksi dataset')


In [ ]:
import os 


try:

  train_directory = '/tmp/rockpaperscissors/rps-cv-images/'
  rock_dir = os.path.join(train_directory,'rock')
  paper_dir = os.path.join(train_directory,'paper')
  scissors_dir = os.path.join(train_directory,'scissors')

  print(os.listdir(train_directory))
  print(os.listdir(train_directory[0:5]))

#directory
  paper_data = os.listdir(paper_dir)
  rock_data = os.listdir(rock_dir)
  scissors_data = os.listdir(scissors_dir)

#Kuantitas
  sub_dir_Qty = len(os.listdir(train_directory))
  paper_Qty = len(os.listdir(paper_dir))
  rock_Qty = len(os.listdir(rock_dir))
  scissor_Qty = len(os.listdir(scissors_dir))


  print(paper_data[:paper_Qty])
  print(rock_data[:rock_Qty])
  print(scissors_data[:scissor_Qty])

except(FileExistsError,FileNotFoundError):
  print('file tidak ditemukan')
  print('coba eksekusi cell diatas')
except:
  print('tidak dapat melist folder')

In [ ]:
#evaluasi data 1
import matplotlib.pyplot as plt


print('total sub directory : ',sub_dir_Qty)
print('total gambar paper :' , paper_Qty)
print('total gambar rock', rock_Qty)
print('total gambar scissor : ', scissor_Qty)
print('keseluruhan : ',paper_Qty+rock_Qty+scissor_Qty)




In [ ]:
#evaluasi data 2
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
%matplotlib inline

picture = 1
try:
    img_paper = [os.path.join(paper_dir,fname)for fname in paper_data[picture-1:picture]]
    img_rock = [os.path.join(rock_dir,fname)for fname in rock_data[picture-1:picture]]
    img_scissor = [os.path.join(scissors_dir,fname)for fname in scissors_data[picture-1:picture]]

  

    for i,img_path in enumerate(img_paper+img_rock+img_scissor):
      
      img = imgplot.imread(img_path)
      plt.figure()
      plt.imshow(img)
      plt.axis('off')
      plt.colorbar()
      plt.show()
except(FileExistsError,FileNotFoundError):
  print('directory tidak ditemukan')
  print('coba eksekusi cell di atas')

except:
  print('terjadi kesalahan')


In [ ]:
#data load and preprocessing
import tensorflow as tf
import numpy as np
import tensorflow.keras.preprocessing.image as img
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator

batch_sizes = 126
img_h = 150
img_w = 150


try:
    train_datagen = ImageDataGenerator(
        rescale = 1./255,
        zca_epsilon=1e-06, 
        rotation_range=40, 
        width_shift_range=0.2,
        height_shift_range=0.2,  
        shear_range=0.2, 
        zoom_range=0.1,
        channel_shift_range=0.0,
        fill_mode = 'nearest',  
        cval=0.0, 
        horizontal_flip=True,  
        validation_split=0.2
    )


    train_set = train_datagen.flow_from_directory(
        train_directory,
        batch_size= batch_sizes,
        shuffle = True,
        target_size = (img_h,img_w),
        color_mode ='rgb',
        class_mode = 'categorical',
        interpolation = 'nearest',
        subset = 'training'

    )

    validation_set = train_datagen.flow_from_directory(
        train_directory,
        batch_size = batch_sizes,
        shuffle = True,
        target_size= (img_h,img_w),
        color_mode = 'rgb',
        class_mode = 'categorical',
        interpolation = 'nearest',
        subset = 'validation'
    )



except:
  print('\nImage Data generator error')
  print('coba eksekusi terlebih dahulu cell di atas\n')


In [ ]:

print('\nTrain dataset')
print(train_set.image_shape)
print(train_set.class_indices)
print(train_set.class_mode)
print('\nvalidation dataset')
print(validation_set.image_shape)
print(validation_set.class_indices)
print(validation_set.class_mode)

In [ ]:
#Training model squential

train_model = tf.keras.models.Sequential([
     #1                                     
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_h,img_w,3)),
    tf.keras.layers.MaxPool2D(2,2),
    #2
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    #3
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    #4
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')                                       
]
)
train_model.summary()

In [ ]:
#COMPILE TRAINING DATA SET
import time
toc = time.perf_counter()



train_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adamax(
                      learning_rate = 0.001,
                      beta_1 = 0.9,
                      beta_2 = 0.999,
                      epsilon = 1e-07,
                      name='Adamax'),
    metrics = ['accuracy']
)




history = train_model.fit(
  train_set,
  epochs = 15,
  steps_per_epoch = 20,
  validation_data = validation_set,
  verbose = 1,
  validation_steps = 3
)

train_model.save('rps.h5')

tcc = time.perf_counter()


In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
vall_loss = history.history['val_loss']

epochs = range(len(accuracy))

time = int(tcc - toc)
jam_pelatihan = 0 if int(time / 3600) < 1 else int(time / 3600)
menit_pelatihan =int(time / 60) if int(time % 60)/60 < 1 else int(time % 60)/60 

plt.plot(epochs,accuracy,'r',label = 'Training Accuracy' )
plt.plot(epochs,val_accuracy,'b',label ='validation Accuracy')
plt.title('Akurasi pada Training dan validation')
plt.legend(loc=0)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

plt.scatter(epochs,loss, label = 'Training Loss')
plt.scatter(epochs,vall_loss, label = 'Validation Loss')
plt.title('Loss pada Training dan Validation')
plt.legend(loc=0)
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()

print('\nWWaktu pelatihan {} jam {} menit.'.format(jam_pelatihan,menit_pelatihan))
print('Atau  {} second'.format(time))

In [ ]:
#keras prediction
import IPython.display as display
import numpy as np
from PIL import  Image
from keras.preprocessing import image
import google.colab.files as files

try:
    upload = files.upload()

    for filename in upload.keys():
      path = filename
      img = image.load_img(path,target_size=(img_h,img_w))
      predict_img = image.img_to_array(img)
      predict_img = np.expand_dims(predict_img,axis=0)

    image_final = np.vstack([predict_img])

  
    prediksi = train_model.predict(image_final,batch_size = 10 )
    print('\nnama file : ',filename)
    if prediksi[0,0] == 1.0 :
      prd = 'paper'
    elif prediksi[0,1] == 1.0 :
      prd = 'rock'
    elif prediksi[0,2] == 1.0 :
      prd = 'scissor'
    else:
      prd = 'prediksi gagal'

    print('Prediksi : {}'.format(prd))  
    IMG = imgplot.imread(path)
    plt.imshow(IMG)
    plt.axis('off')
    plt.show()


except(FileExistsError,FileNotFoundError):
 print('upload file gagal')

except:
  print('terjadi kesalahan')

In [11]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
